<a href="https://colab.research.google.com/github/anagarciasaiz/titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##INICIALIZACIÓN


regresion logistica, arbol de clasificacion, random forest, bayes, gradient boost tree, svm,red neuronal con los datos del titanic


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [814 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Get:13 http://archive.ubun

In [3]:
from pyspark.ml.feature import StringIndexer

In [4]:
spark=SparkSession.builder.appName('clasificacion_titanic').getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/pyspark/titanic.csv" ,header=True, inferSchema=True)
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [7]:
# Tratamiento de valores nulos
# Reemplazar valores nulos en 'Age'  con el valor medio
age_median = data.agg({'Age': 'median'}).first()[0]
data = data.fillna({'Age': age_median})


In [8]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|28.0|    0|    0|      

In [9]:
# Variables categóricas a numéricas
# Convertir 'Sex'a variable numérica indexada
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_num").fit(data)
    for column in ["Sex"]
]
for indexer in indexers:
    data = indexer.transform(data)

In [10]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Sex_num|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|    0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|    1.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|    1.0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|    1.0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|    0.0|
|          6|   

In [11]:
# Eliminar columnas no necesarias y las originales de las variables categóricas
data = data.drop("Sex", "Embarked", "Name", "Ticket", "Cabin","PassengerId","Embarked")

In [12]:
# Seleccionar las columnas necesarias
data = data.select("Survived", "Pclass", "Sex_num", "Age", "SibSp", "Parch", "Fare")

In [13]:
from pyspark.sql.functions import col, count, when


def count_nulls(data):
    # Esta expresión generará una lista de columnas con el recuento de nulos para cada una
    exprs = [count(when(col(c).isNull(), c)).alias(c) for c in data.columns]
    # Aplicar la agregación en todas las columnas
    df_nulls = data.agg(*exprs)
    return df_nulls

# Ejecutar la función y mostrar los resultados
null_counts = count_nulls(data)
null_counts.show()

+--------+------+-------+---+-----+-----+----+
|Survived|Pclass|Sex_num|Age|SibSp|Parch|Fare|
+--------+------+-------+---+-----+-----+----+
|       0|     0|      0|  0|    0|    0|   0|
+--------+------+-------+---+-----+-----+----+



In [14]:
data.printSchema()
data.show(10)

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex_num: double (nullable = false)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)

+--------+------+-------+----+-----+-----+-------+
|Survived|Pclass|Sex_num| Age|SibSp|Parch|   Fare|
+--------+------+-------+----+-----+-----+-------+
|       0|     3|    0.0|22.0|    1|    0|   7.25|
|       1|     1|    1.0|38.0|    1|    0|71.2833|
|       1|     3|    1.0|26.0|    0|    0|  7.925|
|       1|     1|    1.0|35.0|    1|    0|   53.1|
|       0|     3|    0.0|35.0|    0|    0|   8.05|
|       0|     3|    0.0|28.0|    0|    0| 8.4583|
|       0|     1|    0.0|54.0|    0|    0|51.8625|
|       0|     3|    0.0| 2.0|    3|    1| 21.075|
|       1|     3|    1.0|27.0|    0|    2|11.1333|
|       1|     2|    1.0|14.0|    1|    0|30.0708|
+--------+------+-------+----+-----+-----+-------+
only showing

##RANDOM FOREST

In [15]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [17]:
assembler = VectorAssembler(
    inputCols=["Pclass", "Sex_num", "Age", "SibSp", "Parch", "Fare"],
    outputCol="caracteristicas"
)
# Transformar las características
data = assembler.transform(data)

# Escalador para las características
scaler = StandardScaler(inputCol="caracteristicas", outputCol="features_scaled", withStd=True, withMean=False)

# Ajustar el escalador a los datos y transformarlos
scalerModel = scaler.fit(data)
data = scalerModel.transform(data)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = data.randomSplit([0.8, 0.2], seed=53)

In [18]:
# Definir el modelo de Random Forest
rf = RandomForestClassifier(featuresCol="features_scaled", labelCol="Survived", numTrees=10)

# Entrenar el modelo
rfModel = rf.fit(train_data)


In [19]:
# Realizar predicciones en el conjunto de prueba
predictions = rfModel.transform(test_data)

# Evaluador para la precisión
evaluator_precision = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

# Calcular la precisión en el conjunto de datos de prueba
precision = evaluator_precision.evaluate(predictions)
print("Precisión: ", precision)

Precisión:  0.8265895953757225


##ARBOL DE CLASIFICACION

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler,StringIndexer,MinMaxScaler
from pyspark.ml import Pipeline

In [ ]:
assembler = VectorAssembler(
    inputCols=["Pclass","Sex_num", "Age","SibSp","Parch", "Fare"],
    outputCol="caracteristicas_arbol"
)

In [ ]:
scaler=MinMaxScaler(inputCol="caracteristicas_arbol",outputCol="carcateristicas_escaladas")
pipeline=Pipeline(stages=[assembler,scaler])
scalerModel=pipeline.fit(data)
scaledData=scalerModel.transform(data)

data = assembler.transform(data)
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
dc = DecisionTreeClassifier(featuresCol="caracteristicas_arbol", labelCol="Survived")

# Entrena el modelo
dcModel = dc.fit(train_data)

In [ ]:
predictions = dcModel.transform(test_data)


evaluator_precision = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

# Calcular la precisión en el conjunto de datos de prueba
precision = evaluator_precision.evaluate(predictions)
print("Precisión: ", precision)

Precisión:  0.7724137931034483


In [ ]:
predictions.show()

+--------+------+-------+----+-----+-----+--------+--------------------+--------------------+---------------------+-------------+--------------------+----------+
|Survived|Pclass|Sex_num| Age|SibSp|Parch|    Fare|     caracteristicas|     features_scaled|caracteristicas_arbol|rawPrediction|         probability|prediction|
+--------+------+-------+----+-----+-----+--------+--------------------+--------------------+---------------------+-------------+--------------------+----------+
|       0|     1|    0.0|19.0|    3|    2|   263.0|[1.0,0.0,19.0,3.0...|[1.19607032390132...| [1.0,0.0,19.0,3.0...|  [61.0,33.0]|[0.64893617021276...|       0.0|
|       0|     1|    0.0|24.0|    0|    1|247.5208|[1.0,0.0,24.0,0.0...|[1.19607032390132...| [1.0,0.0,24.0,0.0...|  [61.0,33.0]|[0.64893617021276...|       0.0|
|       0|     1|    0.0|28.0|    0|    0|     0.0|(6,[0,2],[1.0,28.0])|(6,[0,2],[1.19607...| (6,[0,2],[1.0,28.0])|  [61.0,33.0]|[0.64893617021276...|       0.0|
|       0|     1|    0.0|28.

##NAIVE BAYES


##REGRESION LOGISTICA

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [ ]:
assembler = VectorAssembler(
    inputCols=["Pclass","Sex_num", "Age","SibSp","Parch", "Fare"],
    outputCol="caracteristicas"
)
data = assembler.transform(data)

# Escalador para las características
scaler = StandardScaler(inputCol="caracteristicas", outputCol="features_scaled", withStd=True, withMean=False)

# Ajustar el escalador a los datos y transformarlos
scalerModel = scaler.fit(data)
data = scalerModel.transform(data)
train_data, test_data = data.randomSplit([0.8, 0.2], seed=53)

In [ ]:
lr = LogisticRegression(featuresCol="features_scaled", labelCol="Survived")

# Entrena el modelo
lrModel = lr.fit(train_data)


In [ ]:
predictions = lrModel.transform(test_data)


evaluator_precision = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

# Calcular la precisión en el conjunto de datos de prueba
precision = evaluator_precision.evaluate(predictions)
print("Precisión: ", precision)

Precisión:  0.7861271676300579


In [ ]:
predictions.show()

+--------+------+-------+----+-----+-----+-------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex_num| Age|SibSp|Parch|   Fare|     caracteristicas|     features_scaled|       rawPrediction|         probability|prediction|
+--------+------+-------+----+-----+-----+-------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1|    0.0|18.0|    1|    0|  108.9|[1.0,0.0,18.0,1.0...|[1.19607032390132...|[-0.4687616755495...|[0.38490938024542...|       1.0|
|       0|     1|    0.0|21.0|    0|    1|77.2875|[1.0,0.0,21.0,0.0...|[1.19607032390132...|[-0.4287025134392...|[0.39443620190171...|       1.0|
|       0|     1|    0.0|28.0|    0|    0|   26.0|[1.0,0.0,28.0,0.0...|[1.19607032390132...|[-0.0660690277054...|[0.48348874876454...|       1.0|
|       0|     1|    0.0|28.0|    0|    0|   39.6|[1.0,0.0,28.0,0.0...|[1.19607032390132...|[-0.1100625011671...|[0.47251211

##GRADIENT BOOST TREE

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
assembler = VectorAssembler(
    inputCols=["Pclass", "Sex_num", "Age", "SibSp", "Parch", "Fare"],
    outputCol="caracteristicas_gbt"
)

data = assembler.transform(data)

scaler = StandardScaler(inputCol="caracteristicas_gbt", outputCol="features_gbt_scaled", withStd=True, withMean=False)
scalerModel = scaler.fit(data)
data = scalerModel.transform(data)

train_data, test_data = data.randomSplit([0.8, 0.2], seed=53)




In [ ]:
# Crear el modelo Gradient-Boosted Trees
gbt = GBTClassifier(featuresCol="features_gbt_scaled", labelCol="Survived", maxIter=10)

# Entrenar el modelo
gbtModel = gbt.fit(train_data)

# Realizar predicciones en el conjunto de prueba
predictions = gbtModel.transform(test_data)


In [ ]:
evaluator_precision = MulticlassClassificationEvaluator(
    labelCol="Survived", predictionCol="prediction", metricName="accuracy"
)

# Calcular la precisión
precision = evaluator_precision.evaluate(predictions)
print("Precisión con GBT: ", precision)


Precisión con GBT:  0.8265895953757225


In [ ]:
predictions.show()

+--------+------+-------+----+-----+-----+-------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex_num| Age|SibSp|Parch|   Fare| caracteristicas_gbt| features_gbt_scaled|       rawPrediction|         probability|prediction|
+--------+------+-------+----+-----+-----+-------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1|    0.0|18.0|    1|    0|  108.9|[1.0,0.0,18.0,1.0...|[1.19607032390132...|[0.33988798191713...|[0.66368869301928...|       0.0|
|       0|     1|    0.0|21.0|    0|    1|77.2875|[1.0,0.0,21.0,0.0...|[1.19607032390132...|[0.45049995700974...|[0.71115494191261...|       0.0|
|       0|     1|    0.0|28.0|    0|    0|   26.0|[1.0,0.0,28.0,0.0...|[1.19607032390132...|[0.97357371023401...|[0.87513526378425...|       0.0|
|       0|     1|    0.0|28.0|    0|    0|   39.6|[1.0,0.0,28.0,0.0...|[1.19607032390132...|[0.32276741200863...|[0.65600354